In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [2]:
appRecord = pd.read_csv("application_record.csv");
creditRecord = pd.read_csv("credit_record.csv");

In [3]:
creditRecord.insert(1,"STATUSINT",creditRecord['STATUS']);

In [4]:
"""
Original Data
0: 1-29 days past due 
1: 30-59 days past due 
2: 60-89 days overdue 
3: 90-119 days overdue 
4: 120-149 days overdue 
5: Overdue or bad debts, write-offs for more than 150 days 
C: paid off that month 
X: No loan for the month
-------------------------
Numerical Encoding
15: 1-29 days past due 
45: 30-59 days past due 
75: 60-89 days overdue 
105: 90-119 days overdue 
135: 120-149 days overdue 
150: Overdue or bad debts, write-offs for more than 150 days 
0: paid off that month 
0: No loan for the month
"""
def statusToInt(status):
    if status == "X": return 0
    elif status == "C": return 0
    elif status == "0": return 15
    elif status == "1": return 45
    elif status == "2": return 75
    elif status == "3": return 105
    elif status == "4": return 145
    elif status == "5": return 150
    else: print("n/a")

In [5]:
creditRecord["STATUSINT"]=creditRecord["STATUS"].map(statusToInt)

In [6]:
creditRecord.insert(1,"TOTALMONTH",creditRecord['MONTHS_BALANCE']);

In [7]:
creditRecord["TOTALMONTH"]=creditRecord["MONTHS_BALANCE"].map(lambda x: -1*x);

In [8]:
maximum = creditRecord.groupby("ID").max()

In [9]:
sums = creditRecord.groupby("ID").sum()

In [10]:
del maximum['STATUSINT']
del maximum['MONTHS_BALANCE']
del maximum['STATUS']
maximum.head()

,TOTALMONTH
ID,
5001711,3
5001712,18
5001713,21
5001714,14
5001715,59


In [11]:
del sums['TOTALMONTH']
del sums['MONTHS_BALANCE']
sums.head()

,STATUSINT
ID,
5001711,45
5001712,150
5001713,0
5001714,0
5001715,0


In [13]:
"""
TOTALMONTH : The duration of the record in the system.
STATUSINT : The total days the person is behind the payment. 
"""
customData = pd.merge(
    maximum,
    sums,
    on="ID",
    how="inner"
)

data = pd.merge(
    appRecord,
    customData,
    on="ID",
    how="inner"
)
data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TOTALMONTH,STATUSINT
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,15,60
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,14,60
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,29,105
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,4,30
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,26,0


In [15]:
# Everything before the last column and the first column is a feature for a crude model. 
X = data.iloc[:data.size,1:19];
X.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,TOTALMONTH
0,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,15
1,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,14
2,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,29
3,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,4
4,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,26


In [16]:
y = data.iloc[:data.size,19:20];
y.head()

,STATUSINT
0,60
1,60
2,105
3,30
4,0


In [17]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
label_encoder = LabelEncoder();
X['CODE_GENDER'] = label_encoder.fit_transform(X['CODE_GENDER']);
X['FLAG_OWN_CAR'] = label_encoder.fit_transform(X['FLAG_OWN_CAR']);
X['FLAG_OWN_REALTY'] = label_encoder.fit_transform(X['FLAG_OWN_REALTY']);
X['NAME_INCOME_TYPE'] = label_encoder.fit_transform(X['NAME_INCOME_TYPE']);
X['NAME_EDUCATION_TYPE'] = label_encoder.fit_transform(X['NAME_EDUCATION_TYPE']);
X['NAME_FAMILY_STATUS'] = label_encoder.fit_transform(X['NAME_FAMILY_STATUS']);
X['NAME_HOUSING_TYPE'] = label_encoder.fit_transform(X['NAME_HOUSING_TYPE']);
"""
Some people don't have occupation information. This will throw a TypeError for the label_encoder. Let's fix this. 
"""
def jobEncodingPrep(status):
    #print(status)
    if pd.isna(status) : return "No Job Info"
    else: return status
X["OCCUPATION_TYPE"]=X["OCCUPATION_TYPE"].map(jobEncodingPrep)
#print(X)

# This is a dumb encoding, as occupation should be one-hot encoded. 
X['OCCUPATION_TYPE'] = label_encoder.fit_transform(X['OCCUPATION_TYPE']);

In [20]:
# Build a model
clf = RandomForestClassifier();
clf.fit(X, np.ravel(y))
print(clf.score(X,np.ravel(y)));

0.9291768384672354


In [21]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [22]:
kf = KFold(n_splits = 10)
kf.get_n_splits(X,y)

10

In [26]:
mse = []
train_scores = []
test_scores = []

for train_index,test_index in kf.split(X):
    X_train,X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,y_test = y.iloc[train_index], y.iloc[test_index]
    
    clfs = RandomForestClassifier();
    clfs.fit(X_train,np.ravel(y_train))
    y_pred = clfs.predict(X_test)
    mse.append(mean_squared_error(y_pred,y_test))
    train_scores.append(clfs.score(X_train,y_train))
    test_scores.append(clfs.score(X_test,y_test))
    

In [27]:
print("MSEs: ", mse)
print("Average MSE: ", np.mean(mse))
print("Train scores: ", train_scores)
print("Averag train sconp.mean(train_scores))
print(test_scores)
print(np.mean(test_scores))

SyntaxError: EOL while scanning string literal (<ipython-input-27-43da37d8fd12>, line 4)

In [28]:
clfs.score(X_train,y_train)

0.9300560770449835